In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from scipy.stats import norm, pearsonr

import os
import time
import itertools

from glove.model3 import *

/home/jcthompson5@ad.wisc.edu/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# import file names
files = os.listdir("data/SET3_Thirdtrial/")
files

['CDanc_CDmono12h_PROVIDET0.csv', 'CDevo_CDmono12h_PROVIDET0.csv']

# fit gLV models

In [5]:
exp_names = []
for file in files:
    # define strain name
    strain = file.split("_")[0]
    
    # import data
    df = pd.read_csv(f"data/SET3_Thirdtrial/{file}")
    df.sort_values(by=["Treatments", "Time"], inplace=True)
    
    # make sure that conditions have at least one measurement
    dfs = []
    for treatment, df_t in df.groupby("Treatments"):
        if df_t.shape[0] > 1:
            dfs.append(df_t)
    df = pd.concat(dfs)

    # determine species names 
    species = df.columns.values[2:]

    # instantiate gLV fit 
    model = gLV(species, df)

    # fit to data 
    model.fit()
    
    # plot parameter distribution
    n_species = len(species)
    Avec = model.params[n_species:n_species + n_species**2]
    Aij_std = np.sqrt(np.diag(model.Ainv))[n_species:n_species + n_species**2]

    plt.figure(figsize=(18,18))
    # set counter for parameter std. 
    k = 0

    for i in range(n_species):
        for j in range(n_species):
            plt.subplot(n_species, n_species, k+1)
            a = np.linspace(Avec[k]-np.std(Avec), Avec[k]+np.std(Avec))
            plt.plot(a, norm.pdf(a,Avec[k],Aij_std[k]))
            plt.axvline(x=0, c='k', alpha=.5)
            k += 1
            if j == 0:
                plt.ylabel(species[i], fontsize=18)
            if i == n_species-1:
                plt.xlabel(species[j], fontsize=18)
            #plt.xlim([-2,2])
    
    plt.suptitle(strain, fontsize=24)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig("params/SET3/"+strain+"_3.pdf")
    plt.close()
    
    # list of parameter names 
    param_names2 = []
    for s1 in species:
        for s2 in species:
            param_names2 += [s1+"*"+s2]

    param_names3 = []
    for i,s1 in enumerate(species):
        for j,s2 in enumerate(species):
            if j != i:
                for k,s3 in enumerate(species):
                    if k > j and k !=i:
                        param_names3 += [s1+"*"+s2+"*"+s3]
            
    param_names = list(species) + param_names2 + param_names3
    
    # compute Wald test for each parameter
    std_errors = np.sqrt(np.diag(model.Ainv))
    walds = model.params/std_errors
    wald_p_vals = 2*norm.cdf(-np.abs(walds))

    # save to df 
    df = pd.DataFrame()
    df["Param name"] = param_names
    df["Param value"] = model.params
    df["Param stdv"]  = np.sqrt(np.diag(model.Ainv))
    df["Param p-value"] = wald_p_vals
    for j, param_name in enumerate(param_names):
        df[param_name]  = model.Ainv[:, j]
    df.to_csv("params/SET3/"+strain+"_3.csv", index=False)

Total samples: 271, Initial regularization: 1.00e-03
Loss: 31.115, Residuals: -0.315
Loss: 16.901, Residuals: 0.175
Loss: 14.077, Residuals: 0.123
Loss: 10.086, Residuals: 0.076
Loss: 7.696, Residuals: 0.053
Loss: 7.149, Residuals: 0.037
Loss: 6.795, Residuals: 0.019
Loss: 6.343, Residuals: 0.030
Loss: 6.011, Residuals: 0.024
Loss: 5.998, Residuals: 0.030
Loss: 5.868, Residuals: 0.028
Loss: 5.633, Residuals: 0.022
Loss: 5.593, Residuals: 0.033
Loss: 5.523, Residuals: 0.033
Loss: 5.394, Residuals: 0.027
Loss: 5.372, Residuals: 0.038
Loss: 5.193, Residuals: 0.027
Loss: 5.189, Residuals: 0.031
Loss: 5.050, Residuals: 0.023
Loss: 5.046, Residuals: 0.027
Loss: 5.040, Residuals: 0.027
Loss: 5.027, Residuals: 0.027
Loss: 4.918, Residuals: 0.021
Loss: 4.916, Residuals: 0.021
Loss: 4.913, Residuals: 0.022
Loss: 4.814, Residuals: 0.017
Loss: 4.813, Residuals: 0.018
Loss: 4.810, Residuals: 0.019
Loss: 4.806, Residuals: 0.020
Loss: 4.762, Residuals: 0.018
Loss: 4.758, Residuals: 0.021
Loss: 4.657,

/tmp/ipykernel_384781/3707771097.py:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[param_name]  = model.Ainv[:, j]
/tmp/ipykernel_384781/3707771097.py:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[param_name]  = model.Ainv[:, j]
/tmp/ipykernel_384781/3707771097.py:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `

Total samples: 225, Initial regularization: 1.00e-03
Loss: 28.595, Residuals: -0.313
Loss: 12.015, Residuals: 0.221
Loss: 8.490, Residuals: 0.010
Loss: 6.073, Residuals: 0.009
Loss: 4.310, Residuals: 0.011
Loss: 4.200, Residuals: 0.014
Loss: 3.992, Residuals: 0.016
Loss: 3.622, Residuals: 0.018
Loss: 3.105, Residuals: 0.016
Loss: 2.612, Residuals: 0.003
Loss: 2.559, Residuals: 0.011
Loss: 2.462, Residuals: 0.011
Loss: 2.302, Residuals: 0.011
Loss: 2.294, Residuals: 0.022
Loss: 2.222, Residuals: 0.020
Loss: 2.089, Residuals: 0.014
Loss: 2.067, Residuals: 0.027
Loss: 2.058, Residuals: 0.028
Loss: 1.975, Residuals: 0.021
Loss: 1.951, Residuals: 0.032
Loss: 1.946, Residuals: 0.031
Loss: 1.903, Residuals: 0.026
Loss: 1.828, Residuals: 0.018
Loss: 1.824, Residuals: 0.024
Loss: 1.791, Residuals: 0.020
Loss: 1.782, Residuals: 0.021
Loss: 1.718, Residuals: 0.011
Loss: 1.716, Residuals: 0.011
Loss: 1.715, Residuals: 0.014
Loss: 1.677, Residuals: 0.010
Loss: 1.676, Residuals: 0.010
Optimization t

/tmp/ipykernel_384781/3707771097.py:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[param_name]  = model.Ainv[:, j]
/tmp/ipykernel_384781/3707771097.py:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[param_name]  = model.Ainv[:, j]
/tmp/ipykernel_384781/3707771097.py:81: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `